In [2]:
pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26297 sha256=4a10fb849c57217e6574bcb9eb4dfb33b7ed38c028626a3f123276cf43b545fc
  Stored in directory: /Users/dhruvrao/Library/Caches/pip/wheels/ab/36/ec/47be2d4e59dc4289e684d5b0dde54d1e72e51a614e57690e85
Successfully built cvzone
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/6e/37/bd9a3cc4d9b8af54fe40a4858ddd7397e163e079dd67eccec7a948f598a9/mediapipe-0.10.21-cp311-cp311-macosx_11_0_universal2.whl.metadata
  Obtaining dependency information for jax from https://files.pythonhosted.org/packages/86/bb/fdc6513a9aada13fd21e9860e2adee5f6eea2b4f0a145b219288875acb26/jax-0.5.3-py3-none-any.whl.metadata
  Obtaining dependency information for jaxlib from https://files.pythonhosted.org/packages/c2/f2/d9397f264141f2289e229b2faf3b3ddb6397b014a09abe234367814f9697/jaxlib-0.5.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/f3/78/b504ca8f7a312918d184e0b8093c62bc9a110d8154f658b591ef5c020d65/opencv_contrib_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata
  Obtaining dependency information for protobuf<5,>=4.25.3 from https://files.pythonhosted.org/packages/b7/03/361e87cc8244

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector

# Capture Video from User Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Make Detector
detector = HandDetector(detectionCon=0.8)
StartDist = None
scale = 0
cx, cy = 500, 500

# Load Image
img1 = cv2.imread('/Users/dhruvrao/Pictures/Uconn/logo.webp')
if img1 is None:
    raise ValueError("Error: Image not found or could not be loaded.")

while True:
    success, img = cap.read()
    if not success:
        print("Failed to read from webcam")
        continue  # Skip this iteration if no frame is captured

    hands, img = detector.findHands(img)

    # Check if 2 hands are detected
    if len(hands) == 2:
        fingers1 = detector.fingersUp(hands[0])
        fingers2 = detector.fingersUp(hands[1])

        print(f"Fingers Up (Hand 1): {fingers1}")
        print(f"Fingers Up (Hand 2): {fingers2}")

        if fingers1 == [1, 1, 0, 0, 0] and fingers2 == [1, 1, 0, 0, 0]:
            lmList1 = hands[0]["lmList"]
            lmList2 = hands[1]["lmList"]

            # Ensure both hands have at least 9 landmarks
            if len(lmList1) > 8 and len(lmList2) > 8:
                p1 = lmList1[8][:2]  # Index finger of hand 1 (x, y)
                p2 = lmList2[8][:2]  # Index finger of hand 2 (x, y)

                # Calculate distance between index fingertips
                if StartDist is None:
                    length, info, img = detector.findDistance(p1, p2, img)
                    StartDist = length

                length, info, img = detector.findDistance(p1, p2, img)
                scale = int((length - StartDist) // 2)
                cx, cy = info[4:]  # Center position between fingers

                print(f"Scale: {scale}, Center: ({cx}, {cy})")

    else:
        StartDist = None  # Reset scale when hands are not detected

    # Resize img1
    h1, w1, _ = img1.shape
    newH, newW = h1 + scale, w1 + scale

    # Ensure new dimensions are even numbers to avoid shape mismatch ( otherwise 1 px will lose )
    newH = max(1, (newH // 2) * 2)
    newW = max(1, (newW // 2) * 2)

    img1_resized = cv2.resize(img1, (newW, newH))

    # Ensure cropping region stays within bounds of img
    h, w, _ = img.shape  # Get frame size

    x1, x2 = max(0, cx - newW // 2), min(w, cx + newW // 2)
    y1, y2 = max(0, cy - newH // 2), min(h, cy + newH // 2)

    # Ensure the img1 overlay fits inside img correctly
    img[y1:y2, x1:x2] = img1_resized[:y2 - y1, :x2 - x1]

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()